In [1]:
import sys
sys.path.append("..")

from utils.evaluation import *
from utils.utils import *

from data import dataset_preprocessing

from utils.evaluation import get_metrics
from xgboost import XGBClassifier, XGBRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge, Lasso

from scipy import stats

import pandas as pd
import numpy as np
import os

import pickle

In [2]:
dataset_name = "OULAD"
mode="cv"
RS=42
hct=10
test_ratio=0.2
val_ratio=0.1
folds=5
target = "binary"
experiment_name = "5CV_paper_final"

### Describe raw data

In [3]:
assessments_df = pd.read_csv(f'../data/raw/{dataset_name}/assessments.csv')
courses_df = pd.read_csv(f'../data/raw/{dataset_name}/courses.csv')
studentAssessment_df = pd.read_csv(f'../data/raw/{dataset_name}/studentAssessment.csv')
studentInfo_df = pd.read_csv(f'../data/raw/{dataset_name}/studentInfo.csv')
studentRegistration_df = pd.read_csv(f'../data/raw/{dataset_name}/studentRegistration.csv')
studentVle_df = pd.read_csv(f'../data/raw/{dataset_name}/studentVle.csv')
vle_df = pd.read_csv(f'../data/raw/{dataset_name}/vle.csv')


In [4]:
#########  Preprocessing #################################
if not os.path.exists(f"../data/prepared/{dataset_name}/df_prepared.pickle"):

    assessments_df = pd.read_csv(f'../data/raw/{dataset_name}/assessments.csv')
    courses_df = pd.read_csv(f'../data/raw/{dataset_name}/courses.csv')
    studentAssessment_df = pd.read_csv(f'../data/raw/{dataset_name}/studentAssessment.csv')
    studentInfo_df = pd.read_csv(f'../data/raw/{dataset_name}/studentInfo.csv')
    studentRegistration_df = pd.read_csv(f'../data/raw/{dataset_name}/studentRegistration.csv')
    studentVle_df = pd.read_csv(f'../data/raw/{dataset_name}/studentVle.csv')
    vle_df = pd.read_csv(f'../data/raw/{dataset_name}/vle.csv')


    # Remove all withdrawn
    studentInfo_df = studentInfo_df.loc[studentInfo_df.final_result!="Withdrawn"]
    studentInfo_df.shape

    # Assessment performance features
    merged_assessments_df = pd.merge(studentAssessment_df,assessments_df,on="id_assessment")

    avg_tma = [merged_assessments_df.loc[np.logical_and(merged_assessments_df.id_student==i,merged_assessments_df.assessment_type=="TMA"),"score"].mean() for i in studentInfo_df.id_student.values]
    avg_cma = [merged_assessments_df.loc[np.logical_and(merged_assessments_df.id_student==i,merged_assessments_df.assessment_type=="CMA"),"score"].mean() for i in studentInfo_df.id_student.values]
    avg_exam = [merged_assessments_df.loc[np.logical_and(merged_assessments_df.id_student==i,merged_assessments_df.assessment_type=="Exam"),"score"].mean() for i in studentInfo_df.id_student.values]

    studentInfo_df["avg_tma"] = avg_tma
    studentInfo_df["avg_cma"] = avg_cma
    studentInfo_df["avg_exam"] = avg_exam

    # Get VLE features
    vle_merged = pd.merge(studentVle_df,vle_df,on=["code_module", "code_presentation", "id_site"])
    for activity_type in vle_df.activity_type.unique():
        agg = vle_merged.loc[vle_merged.activity_type==activity_type].groupby("id_student")
        count_click_dict = dict(agg.count()["sum_click"])
        sum_click_dict = dict(agg.sum()["sum_click"])
        studentInfo_df[f"Count_Visits_{activity_type}"] = studentInfo_df["id_student"].apply(lambda x: sum_click_dict[x] if x in count_click_dict.keys() else 0)
        studentInfo_df[f"Sum_Clicks_{activity_type}"] = studentInfo_df["id_student"].apply(lambda x: sum_click_dict[x] if x in sum_click_dict.keys() else 0)

    with open(f"../data/prepared/{dataset_name}/df_prepared.pickle", 'wb') as handle:
        pickle.dump(studentInfo_df, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
    df = studentInfo_df

else:    
    with open(f"../data/prepared/{dataset_name}/df_prepared.pickle", 'rb') as handle:
        df = pickle.load(handle)


In [5]:
# # Remove all withdrawn
# studentInfo_df = studentInfo_df.loc[studentInfo_df.final_result!="Withdrawn"]
# studentInfo_df.shape

In [6]:
# # Get final score
# merged_assessments_df = pd.merge(studentAssessment_df,assessments_df,on="id_assessment")
# final_results = [merged_assessments_df.loc[merged_assessments_df.id_student==i,"score"].dot(merged_assessments_df.loc[merged_assessments_df.id_student==i,"weight"]/100) for i in studentInfo_df.id_student.values]
# studentInfo_df["final_score"] = final_results

In [7]:
# Exclude those with 0 final score who did not fail (three from differen scores as error, and code module GGG as 0 is normal for it)
# studentInfo_df = studentInfo_df.loc[~np.logical_and(studentInfo_df["final_score"]<1, studentInfo_df["final_result"]!="Fail")]

In [8]:
# Get Avg_TMA_Score, Avg_CMA_Score, Exam_Score
# avg_tma = [merged_assessments_df.loc[np.logical_and(merged_assessments_df.id_student==i,merged_assessments_df.assessment_type=="TMA"),"score"].dot(merged_assessments_df.loc[np.logical_and(merged_assessments_df.id_student==i,merged_assessments_df.assessment_type=="TMA"),"weight"]/100) for i in studentInfo_df.id_student.values[:100]]


In [9]:
# # Assessment performance features
# merged_assessments_df = pd.merge(studentAssessment_df,assessments_df,on="id_assessment")

# avg_tma = [merged_assessments_df.loc[np.logical_and(merged_assessments_df.id_student==i,merged_assessments_df.assessment_type=="TMA"),"score"].mean() for i in studentInfo_df.id_student.values]
# avg_cma = [merged_assessments_df.loc[np.logical_and(merged_assessments_df.id_student==i,merged_assessments_df.assessment_type=="CMA"),"score"].mean() for i in studentInfo_df.id_student.values]
# avg_exam = [merged_assessments_df.loc[np.logical_and(merged_assessments_df.id_student==i,merged_assessments_df.assessment_type=="Exam"),"score"].mean() for i in studentInfo_df.id_student.values]

# studentInfo_df["avg_tma"] = avg_tma
# studentInfo_df["avg_cma"] = avg_cma
# studentInfo_df["avg_exam"] = avg_exam

In [10]:
# # Get VLE features
# vle_merged = pd.merge(studentVle_df,vle_df,on=["code_module", "code_presentation", "id_site"])
# for activity_type in vle_df.activity_type.unique():
#     agg = vle_merged.loc[vle_merged.activity_type==activity_type].groupby("id_student")
#     count_click_dict = dict(agg.count()["sum_click"])
#     sum_click_dict = dict(agg.sum()["sum_click"])
#     studentInfo_df[f"Count_Visits_{activity_type}"] = studentInfo_df["id_student"].apply(lambda x: sum_click_dict[x] if x in count_click_dict.keys() else 0)
#     studentInfo_df[f"Sum_Clicks_{activity_type}"] = studentInfo_df["id_student"].apply(lambda x: sum_click_dict[x] if x in sum_click_dict.keys() else 0)

In [11]:
# df = studentInfo_df.copy()

In [12]:
df = df.drop(["id_student", "code_module", "code_presentation", "avg_exam"],axis=1)

In [13]:
df["final_result"] = df["final_result"].apply(lambda x: 0 if x=="Fail" else 1)

In [14]:
y_col = "final_result"
demographic_cols = ['gender', 'region', 'imd_band', 'age_band', 'disability', 'highest_education']
perf_cols = ["avg_cma", "avg_tma", 'num_of_prev_attempts', 'studied_credits']
activity_cols = [i for i in df.columns if "Sum_Clicks" in i] + [i for i in df.columns if "Count_Visits" in i]
other_cols = []

set(df.columns)-set([y_col]+demographic_cols+perf_cols+activity_cols+other_cols)

set()

In [15]:
desc_df_dict = {"No. of samples": df.shape[0],
           "No. of features": df.shape[1],
           "Performance features": len(perf_cols),
           "Demographic features": len(demographic_cols),
           "Activity features": len(activity_cols),
           "Other features": len(other_cols),
           "Categorical features": len(df.columns[list(np.logical_and(df.nunique() > 2, df.dtypes == "object"))]),     
           "Total cardinality": df[df.columns[list(np.logical_and(df.nunique() > 2, df.dtypes == "object"))]].nunique().sum(),     
           "% NA": df.isna().sum().sum()/sum(df.shape),
           "Target $\textbf{y} \in$": f"[1..{df[y_col].nunique()}]",
#            "High cardinality levels":  list(df.loc[:,list(df.columns[list(np.logical_and(df.nunique() >= 10, df.dtypes == "object"))])].nunique().sort_values().values),
          
}
desc_df = pd.DataFrame([desc_df_dict],index=["cortez"])
desc_df

,No. of samples,No. of features,Performance features,Demographic features,Activity features,Other features,Categorical features,Total cardinality,% NA,Target $\textbf{y} \in$
cortez,22437,51,4,6,40,0,4,31,0.480034,[1..2]


In [16]:
print(desc_df.transpose().to_latex())

\begin{tabular}{ll}
\toprule
{} &    cortez \\
\midrule
No. of samples          &     22437 \\
No. of features         &        51 \\
Performance features    &         4 \\
Demographic features    &         6 \\
Activity features       &        40 \\
Other features          &         0 \\
Categorical features    &         4 \\
Total cardinality       &        31 \\
\% NA                    &  0.480034 \\
Target \$\textbackslash textbf\{y\} \textbackslash in\$ &    [1..2] \\
\bottomrule
\end{tabular}



### Preprocessing and preparation

In [17]:
data_path = f"{mode}_RS{RS}_hct{hct}"
if mode == "cv":
    data_path += f"_{folds}folds"
elif mode == "train_test":
    data_path += f"_split{1-test_ratio*100}-{test_ratio*100}"
elif mode == "train_val_test":
    data_path += f"_split{round(100-(test_ratio+val_ratio)*100)}-{round(test_ratio*100)}-{round(val_ratio*100)}"


# If no data_dict for the configuration exists, run preprocessing, else load data_dict
if not os.path.exists(f"../data/prepared/{dataset_name}/"+data_path+"/data_dict.pickle"):
    dataset_preprocessing.process_dataset(dataset_name, target, mode, RS, hct, test_ratio, val_ratio, folds)
with open(f"../data/prepared/{dataset_name}/{data_path}/data_dict.pickle", 'rb') as handle:
        data_dict = pickle.load(handle)


## Evaluation of categorical data treatment methods

In [18]:
conditions = ["ignore", "ohe", "target", "ordinal", "catboost", "glmm"]

In [19]:
early_stopping_rounds = 10
max_evals = 5

In [27]:
if not os.path.exists(f"../results/{dataset_name}/{experiment_name}/results_encodings.pickle"):

    results_encodings = {}
    results_encodings_feature_importances = {}

    for fold in range(folds):
        results_encodings[fold] = {}
        results_encodings_feature_importances[fold] = {}
        # Create baseline
        y_train = data_dict[f"y_train_{fold}"]
        y_val = data_dict[f"y_val_{fold}"]
        y_test = data_dict[f"y_test_{fold}"]
        y_train_val = np.concatenate([y_train,y_val])

        max_ = np.argmax(np.unique(y_train_val,return_counts=True)[1])
        y_train_val_pred_base = np.ones(y_train_val.shape[0])*max_
        y_test_pred_base = np.ones(y_test.shape[0])*max_

        results_encodings[fold]["Baseline"] = {}
        eval_res_train = get_metrics(y_train_val, y_train_val_pred_base, target=target)
        for metric in eval_res_train.keys():
            results_encodings[fold]["Baseline"][metric + " Train"] = eval_res_train[metric]
        eval_res_test = get_metrics(y_test, y_test_pred_base, target=target)
        for metric in eval_res_test.keys():
            results_encodings[fold]["Baseline"][metric + " Test"] = eval_res_test[metric]

        for condition in conditions:
            print(f"Preparing results for fold {fold}, condition={condition}")
            # Retrieve data
            z_cols = data_dict["z_cols"]

            X_train = data_dict[f"X_train_{fold}"]
            y_train = data_dict[f"y_train_{fold}"]

            X_val = data_dict[f"X_val_{fold}"]
            y_val = data_dict[f"y_val_{fold}"]

            X_test = data_dict[f"X_test_{fold}"]
            y_test = data_dict[f"y_test_{fold}"]

    ## ALL BUT PERFORMANCE:
            # Define data subset for evaluation
    #         X_train = X_train[[i for i in X_train.columns if all([j not in i for j in perf_cols])]]
    #         X_val = X_val[[i for i in X_val.columns if all([j not in i for j in perf_cols])]]
    #         X_test = X_test[[i for i in X_test.columns if all([j not in i for j in perf_cols])]]

            # Define condition data subset
    #         if condition != "ignore":
    #             z_encoded_train = data_dict[f"z_{condition}_encoded_train_{fold}"] 
    #             z_encoded_val = data_dict[f"z_{condition}_encoded_val_{fold}"] 
    #             z_encoded_test = data_dict[f"z_{condition}_encoded_test_{fold}"] 

    #             X_train = pd.concat([X_train,z_encoded_train],axis=1)
    #             X_val = pd.concat([X_val,z_encoded_val],axis=1)
    #             X_test = pd.concat([X_test,z_encoded_test],axis=1)

    # ALL BUT PERFORMANCE & ACTIVITY:
    #         Define data subset for evaluation
            X_train = X_train[[i for i in X_train.columns if all([j not in i for j in perf_cols+activity_cols])]]
            X_val = X_val[[i for i in X_val.columns if all([j not in i for j in perf_cols+activity_cols])]]
            X_test = X_test[[i for i in X_test.columns if all([j not in i for j in perf_cols+activity_cols])]]

    #         Define condition data subset
            if condition != "ignore":
                z_encoded_train = data_dict[f"z_{condition}_encoded_train_{fold}"] 
                z_encoded_val = data_dict[f"z_{condition}_encoded_val_{fold}"] 
                z_encoded_test = data_dict[f"z_{condition}_encoded_test_{fold}"] 

                X_train = pd.concat([X_train,z_encoded_train],axis=1)
                X_val = pd.concat([X_val,z_encoded_val],axis=1)
                X_test = pd.concat([X_test,z_encoded_test],axis=1)


    ## ONLY CATEGORICAL: --> Produces trash as almost never better than baseline
    #         if condition != "ignore":        
    #             X_train = data_dict[f"z_{condition}_encoded_train_{fold}"] 
    #             X_val = data_dict[f"z_{condition}_encoded_val_{fold}"] 
    #             X_test = data_dict[f"z_{condition}_encoded_test_{fold}"] 

    #         else:
    #             continue

            X_train_val = pd.concat([X_train,X_val])
            y_train_val = np.concatenate([y_train,y_val])

            # Train base models
            res, feats = evaluate_logreg(X_train_val, y_train_val, X_test, y_test, target=target,tune=False, seed=RS)
            results_encodings[fold]["LR_"+condition] = res
            results_encodings_feature_importances[fold]["LR_"+condition] = feats

            res, feats = evaluate_xgb(X_train_val, y_train_val, X_test, y_test, target, tune=False, max_evals=max_evals, early_stopping_rounds=early_stopping_rounds, seed=RS)
            results_encodings[fold]["XGB_"+condition] = res
            results_encodings_feature_importances[fold]["XGB_"+condition] = feats

            # Train tuned models
            res, feats = evaluate_logreg(X_train_val, y_train_val, X_test, y_test, target=target, max_evals=max_evals, tune=True, seed=RS)
            results_encodings[fold]["LR_"+condition+"_tuned"] = res
            results_encodings_feature_importances[fold]["LR_"+condition+"_tuned"] = feats

            res, feats = evaluate_xgb(X_train_val, y_train_val, X_test, y_test, target, tune=True, max_evals=max_evals, early_stopping_rounds=early_stopping_rounds, seed=RS)
            results_encodings[fold]["XGB_"+condition+"_tuned"] = res
            results_encodings_feature_importances[fold]["XGB_"+condition+"_tuned"] = feats
    
    if not os.path.exists(f"../results/{dataset_name}/{experiment_name}"):
        os.makedirs(f"../results/{dataset_name}/{experiment_name}")
    with open(f"../results/{dataset_name}/{experiment_name}/results_encodings.pickle", 'wb') as handle:
        pickle.dump(results_encodings, handle, protocol=pickle.HIGHEST_PROTOCOL)
    with open(f"../results/{dataset_name}/{experiment_name}/results_encodings_feature_importances.pickle", 'wb') as handle:
        pickle.dump(results_encodings_feature_importances, handle, protocol=pickle.HIGHEST_PROTOCOL)

else:
    with open(f"../results/{dataset_name}/{experiment_name}/results_encodings.pickle", 'rb') as handle:
        results_encodings = pickle.load(handle)
    with open(f"../results/{dataset_name}/{experiment_name}/results_encodings_feature_importances.pickle", 'rb') as handle:
        results_encodings_feature_importances = pickle.load(handle)
        
        
results_encodings_df = pd.DataFrame(results_encodings[0]).transpose().sort_values("F1 Test",ascending=False).round(4)
results_encodings_df[["Accuracy Train", "F1 Train", "AUROC Train", "Accuracy Test", "F1 Test", "AUROC Test"]].style.highlight_max(color = 'lightgreen', axis = 0)

Preparing results for fold 0, condition=ignore
SCORE: 0.6236650205401243                                                                                              
SCORE: 0.6236651971377892                                                                                              
SCORE: 0.6236653496493902                                                                                              
SCORE: 0.6236628137576451                                                                                              
SCORE: 0.6236653495420327                                                                                              
100%|██████████████████████████████████████████████████| 5/5 [00:00<00:00, 16.13trial/s, best loss: 0.6236628137576451]
The best hyperparameters are :  

{'C': 0.07433411394132879}
Default performance on Test: 0.6141019933315486
SCORE: 0.6476721883883189                                                                                              
SCOR

100%|███████████████████████████████████████████████████| 5/5 [00:05<00:00,  1.18s/trial, best loss: 0.599658001737366]
The best hyperparameters after step 3 are :  

{'learning_rate': 0.23270078553187346, 'n_estimators': 459.0, 'seed': 0, 'max_depth': 4.0, 'min_child_weight': 7.0, 'colsample_bytree': 0.7352852497514042, 'subsample': 0.6386337481887971}
Test Performance after third tuning round: 0.6063132838297732
SCORE: 0.6024875039292996                                                                                              
SCORE: 0.600648704907919                                                                                               
SCORE: 0.6014805410581645                                                                                              
SCORE: 0.6015721827374974                                                                                              
SCORE: 0.6021853481218435                                                                             

SCORE: 0.6257396740400203                                                                                              
SCORE: 0.604482932656958                                                                                               
SCORE: 0.6034237539794147                                                                                              
SCORE: 0.6372387800116414                                                                                              
SCORE: 0.6006459402163772                                                                                              
100%|██████████████████████████████████████████████████| 5/5 [00:03<00:00,  1.52trial/s, best loss: 0.6006459402163772]
The best hyperparameters after step 2 are :  

{'learning_rate': 0.36680921045468584, 'n_estimators': 268.0, 'seed': 0, 'max_depth': 4.0, 'min_child_weight': 3.0}
Test Performance after second tuning round: 0.6049853436292145
SCORE: 0.5999695149287458                             

SCORE: 0.5997530759292096                                                                                              
SCORE: 0.5997536503107777                                                                                              
SCORE: 0.6013928622595787                                                                                              
100%|██████████████████████████████████████████████████| 5/5 [00:00<00:00,  9.78trial/s, best loss: 0.5997529508469788]
The best hyperparameters are :  

{'C': 0.9873092146842414}
Default performance on Test: 0.63158157263424
SCORE: 0.6204294106552372                                                                                              
SCORE: 0.6315750337382251                                                                                              
SCORE: 0.6115764431004491                                                                                              
SCORE: 0.6323261827220732                             

SCORE: 0.6241185740687458                                                                                              
SCORE: 0.6211326671126494                                                                                              
SCORE: 0.6211154953834385                                                                                              
SCORE: 0.6241003152697718                                                                                              
SCORE: 0.6253903228585348                                                                                              
100%|██████████████████████████████████████████████████| 5/5 [00:01<00:00,  3.27trial/s, best loss: 0.6211154953834385]
The best hyperparameters are :  

{'learning_rate': 0.4797345657644125, 'n_estimators': 409.0, 'seed': 0, 'max_depth': 8.0, 'min_child_weight': 5.0, 'colsample_bytree': 0.6394105907473081, 'subsample': 0.676882772727635, 'gamma': 2.546426874549293, 'reg_alpha': 4.0, 'reg_lambda':

...

SCORE: 0.5971789368910974                                                                                              
SCORE: 0.5974459680726518                                                                                              
SCORE: 0.5973272071381414                                                                                              
SCORE: 0.5974890676271228                                                                                              
SCORE: 0.597631443351963                                                                                               
100%|██████████████████████████████████████████████████| 5/5 [00:03<00:00,  1.29trial/s, best loss: 0.5971789368910974]
The best hyperparameters are :  

{'learning_rate': 0.3251950373433666, 'n_estimators': 197.0, 'seed': 0, 'max_depth': 2.0, 'min_child_weight': 1.0, 'colsample_bytree': 0.5608440465303586, 'subsample': 0.682997879073356, 'gamma': 4.339278940523746, 'reg_alpha': 0.0, 'reg_lambda':

SCORE: 0.6525158143422415                                                                                              
100%|██████████████████████████████████████████████████| 5/5 [00:13<00:00,  2.74s/trial, best loss: 0.6027323462265183]
The best hyperparameters after step 2 are :  

{'learning_rate': 0.05453663646373698, 'n_estimators': 92.0, 'seed': 0, 'max_depth': 11.0, 'min_child_weight': 10.0}
Test Performance after second tuning round: 0.6022300941487789
SCORE: 0.6143935909840421                                                                                              
SCORE: 0.6127552675726959                                                                                              
SCORE: 0.6213931848348881                                                                                              
SCORE: 0.6252742300442466                                                                                              
SCORE: 0.639421880846931                             

Default performance on Test: 0.6178878935579406
SCORE: 0.6033682178101658                                                                                              
SCORE: 0.6212524804231888                                                                                              
SCORE: 0.6021776168226325                                                                                              
SCORE: 0.6695310329967695                                                                                              
SCORE: 0.6054339829733932                                                                                              
100%|██████████████████████████████████████████████████| 5/5 [00:07<00:00,  1.44s/trial, best loss: 0.6021776168226325]
The best hyperparameters after step 1  are :  

{'learning_rate': 0.26243517658424165, 'n_estimators': 389.0}
Test Performance after first tuning round: 0.6365859267071927
SCORE: 0.6002202950888103                                   

SCORE: 0.6008890133054804                                                                                              
SCORE: 0.6015546042291552                                                                                              
100%|██████████████████████████████████████████████████| 5/5 [00:04<00:00,  1.12trial/s, best loss: 0.6008890133054804]
The best hyperparameters are :  

{'learning_rate': 0.3453067995543792, 'n_estimators': 190.0, 'seed': 0, 'max_depth': 1.0, 'min_child_weight': 1.0, 'colsample_bytree': 0.8161299501447582, 'subsample': 0.9108247493188844, 'gamma': 1.7351540675723383, 'reg_alpha': 9.0, 'reg_lambda': 1.2725650194696203}
Test Performance after last tuning round: 0.6007589863075742
Preparing results for fold 4, condition=glmm
SCORE: 0.5992873061595962                                                                                              
SCORE: 0.5987683195379809                                                                                      

,Accuracy Train,F1 Train,AUROC Train,Accuracy Test,F1 Test,AUROC Test
LR_ignore,0.683000,0.811700,0.521200,0.696300,0.821000,0.516700
XGB_ignore,0.683000,0.811700,0.521200,0.696300,0.821000,0.516700
LR_ordinal_tuned,0.683000,0.811700,0.560000,0.696300,0.821000,0.546100
LR_ordinal,0.683000,0.811700,0.560000,0.696300,0.821000,0.546100
LR_ignore_tuned,0.683000,0.811700,0.521200,0.696300,0.821000,0.516700
XGB_ignore_tuned,0.683000,0.811700,0.521200,0.696300,0.821000,0.516700
LR_catboost_tuned,0.684500,0.809400,0.625800,0.695400,0.818100,0.613800
LR_catboost,0.684600,0.809400,0.625800,0.695400,0.818100,0.613800
XGB_ohe_tuned,0.689700,0.809000,0.645200,0.697900,0.816100,0.633800
LR_ohe,0.689800,0.808100,0.641300,0.699600,0.815900,0.633100


In [41]:
results_encodings_df = pd.DataFrame(results_encodings[1]).transpose().sort_values("F1 Test",ascending=False).round(4)
results_encodings_df[["Accuracy Train", "F1 Train", "AUROC Train", "Accuracy Test", "F1 Test", "AUROC Test"]].style.highlight_max(color = 'lightgreen', axis = 0)

,Accuracy Train,F1 Train,AUROC Train,Accuracy Test,F1 Test,AUROC Test
LR_ignore,0.687100,0.814500,0.521100,0.680300,0.809700,0.517400
XGB_ignore_tuned,0.687100,0.814500,0.521100,0.680300,0.809700,0.517400
XGB_ignore,0.687100,0.814500,0.521100,0.680300,0.809700,0.517400
Baseline,0.687100,0.814500,0.500000,0.680300,0.809700,0.500000
LR_ignore_tuned,0.687100,0.814500,0.521100,0.680300,0.809700,0.517400
XGB_ordinal_tuned,0.692900,0.812400,0.646200,0.689200,0.809400,0.646100
LR_target_tuned,0.688800,0.810900,0.637100,0.687800,0.809000,0.639200
LR_catboost_tuned,0.686600,0.811500,0.627600,0.688900,0.809000,0.623700
LR_catboost,0.686900,0.811700,0.627600,0.688700,0.808800,0.623800
LR_ohe,0.691700,0.811000,0.639000,0.689600,0.808600,0.643500


In [42]:
models = results_encodings[0].keys()
metric = "F1 Test"

encodings_folds_df = pd.DataFrame([pd.DataFrame(results_encodings[fold_num]).loc[metric,models] for fold_num in results_encodings.keys()],index=results_encodings.keys())
encodings_mean_df = encodings_folds_df.mean(axis=0)
encodings_std_df = encodings_folds_df.std(axis=0)

methods = sorted(list(encodings_mean_df.index))[1:]
not_tuned = ["Baseline"]+methods[::2]
tuned = ["Baseline"]+methods[1::2]

res_df_tune_comp_mean = pd.DataFrame([encodings_mean_df.loc[not_tuned].values,encodings_mean_df.loc[tuned]],index=["Untuned","Tuned"],columns=not_tuned).transpose()
res_df_tune_comp_std = pd.DataFrame([encodings_std_df.loc[not_tuned].values,encodings_std_df.loc[tuned]],index=["Untuned","Tuned"],columns=not_tuned).transpose()

res_df_tune_comp_mean.round(2).style.highlight_max(color = 'lightgreen', axis = 1)

,Untuned,Tuned
Baseline,0.810000,0.810000
LR_catboost,0.810000,0.810000
LR_glmm,0.810000,0.810000
LR_ignore,0.810000,0.810000
LR_ohe,0.810000,0.810000
LR_ordinal,0.810000,0.810000
LR_target,0.810000,0.810000
XGB_catboost,0.790000,0.810000
XGB_glmm,0.780000,0.810000
XGB_ignore,0.810000,0.810000


In [43]:
latex_df = res_df_tune_comp_mean.round(2).astype(str) + " (" +  res_df_tune_comp_std.round(3).astype(str) + ")"
latex_df

,Untuned,Tuned
Baseline,0.81 (0.005),0.81 (0.005)
LR_catboost,0.81 (0.006),0.81 (0.006)
LR_glmm,0.81 (0.004),0.81 (0.004)
LR_ignore,0.81 (0.005),0.81 (0.005)
LR_ohe,0.81 (0.004),0.81 (0.004)
LR_ordinal,0.81 (0.005),0.81 (0.005)
LR_target,0.81 (0.004),0.81 (0.004)
XGB_catboost,0.79 (0.005),0.81 (0.006)
XGB_glmm,0.78 (0.01),0.81 (0.004)
XGB_ignore,0.81 (0.005),0.81 (0.005)


In [44]:
print(latex_df.to_latex())

\begin{tabular}{lll}
\toprule
{} &       Untuned &         Tuned \\
\midrule
Baseline     &  0.81 (0.005) &  0.81 (0.005) \\
LR\_catboost  &  0.81 (0.006) &  0.81 (0.006) \\
LR\_glmm      &  0.81 (0.004) &  0.81 (0.004) \\
LR\_ignore    &  0.81 (0.005) &  0.81 (0.005) \\
LR\_ohe       &  0.81 (0.004) &  0.81 (0.004) \\
LR\_ordinal   &  0.81 (0.005) &  0.81 (0.005) \\
LR\_target    &  0.81 (0.004) &  0.81 (0.004) \\
XGB\_catboost &  0.79 (0.005) &  0.81 (0.006) \\
XGB\_glmm     &   0.78 (0.01) &  0.81 (0.004) \\
XGB\_ignore   &  0.81 (0.005) &  0.81 (0.005) \\
XGB\_ohe      &   0.8 (0.004) &  0.81 (0.006) \\
XGB\_ordinal  &   0.8 (0.003) &  0.81 (0.005) \\
XGB\_target   &   0.8 (0.006) &  0.81 (0.004) \\
\bottomrule
\end{tabular}



### Performance Comparison

In [45]:
# For LR
models = ["Baseline"]+[i for i in results_encodings[0].keys() if ("tuned" in i and "LR" in i)]
metric = "F1 Test"

#####
dataset_res_dict = {}
best_models = {}
t_test_results = {}

use_df = pd.DataFrame([pd.DataFrame(results_encodings[fold_num]).loc[metric,models] for fold_num in results_encodings.keys()],index=results_encodings.keys())

df_mean = pd.DataFrame((use_df).mean(axis=0).round(3).astype(str) + " (" + use_df.std(axis=0).round(3).astype(str) + ")").transpose()
model_dict = {i: df_mean[i].values[0] for i in df_mean.columns}

best_model = use_df.columns[use_df.mean(axis=0).argmax()]

t_test_res = np.array([stats.ttest_rel(use_df[best_model].values, use_df[model].values)[1] for model in models]).round(3)
t_test_res[np.isnan(t_test_res)] = 1.
    
res_df_lr = pd.DataFrame([model_dict])

def negative_bold(val):
    i = np.where(val.name==np.array(models))[0][0]
    return ["font-weight: bold"  if t_test_res[i]>=0.05 else "" for dataset_name in val.keys()]
    # Case without transpose:
#     return ["font-weight: bold"  if t_test_results[val.name][i]>=0.05 else "" for i in range(len(val))]

res_df_lr.style.apply(negative_bold)


,Baseline,LR_ignore_tuned,LR_ohe_tuned,LR_target_tuned,LR_ordinal_tuned,LR_catboost_tuned,LR_glmm_tuned
0,0.814 (0.005),0.814 (0.005),0.81 (0.004),0.81 (0.004),0.813 (0.005),0.81 (0.006),0.809 (0.004)


In [46]:
# For LR
models = ["Baseline"]+[i for i in results_encodings[0].keys() if ("tuned" in i and "XGB" in i)]
metric = "F1 Test"

#####
dataset_res_dict = {}
best_models = {}
t_test_results = {}

use_df = pd.DataFrame([pd.DataFrame(results_encodings[fold_num]).loc[metric,models] for fold_num in results_encodings.keys()],index=results_encodings.keys())

df_mean = pd.DataFrame((use_df).mean(axis=0).round(3).astype(str) + " (" + use_df.std(axis=0).round(3).astype(str) + ")").transpose()
model_dict = {i: df_mean[i].values[0] for i in df_mean.columns}

best_model = use_df.columns[use_df.mean(axis=0).argmax()]

t_test_res = np.array([stats.ttest_rel(use_df[best_model].values, use_df[model].values)[1] for model in models]).round(3)
t_test_res[np.isnan(t_test_res)] = 1.
    
res_df_xgb = pd.DataFrame([model_dict])

def negative_bold(val):
    i = np.where(val.name==np.array(models))[0][0]
    return ["font-weight: bold"  if t_test_res[i]>=0.05 else "" for dataset_name in val.keys()]
    # Case without transpose:
#     return ["font-weight: bold"  if t_test_results[val.name][i]>=0.05 else "" for i in range(len(val))]

res_df_xgb.style.apply(negative_bold)


,Baseline,XGB_ignore_tuned,XGB_ohe_tuned,XGB_target_tuned,XGB_ordinal_tuned,XGB_catboost_tuned,XGB_glmm_tuned
0,0.814 (0.005),0.814 (0.005),0.808 (0.006),0.808 (0.004),0.809 (0.005),0.808 (0.006),0.807 (0.004)


In [47]:
res_df_lr.columns = [i.split("_")[1] if i != "Baseline" else "Baseline" for i in res_df_lr.columns]    
res_df_xgb.columns = [i.split("_")[1] if i != "Baseline" else "Baseline" for i in res_df_xgb.columns]    

latex_df_encodings = pd.concat([res_df_lr,res_df_xgb],axis=0)
latex_df_encodings.index = ["LR", "XGB"]
latex_df_encodings

,Baseline,ignore,ohe,target,ordinal,catboost,glmm
LR,0.814 (0.005),0.814 (0.005),0.81 (0.004),0.81 (0.004),0.813 (0.005),0.81 (0.006),0.809 (0.004)
XGB,0.814 (0.005),0.814 (0.005),0.808 (0.006),0.808 (0.004),0.809 (0.005),0.808 (0.006),0.807 (0.004)


In [48]:
print(latex_df_encodings.round(2).to_latex())


\begin{tabular}{llllllll}
\toprule
{} &       Baseline &         ignore &            ohe &         target &        ordinal &       catboost &           glmm \\
\midrule
LR  &  0.814 (0.005) &  0.814 (0.005) &   0.81 (0.004) &   0.81 (0.004) &  0.813 (0.005) &   0.81 (0.006) &  0.809 (0.004) \\
XGB &  0.814 (0.005) &  0.814 (0.005) &  0.808 (0.006) &  0.808 (0.004) &  0.809 (0.005) &  0.808 (0.006) &  0.807 (0.004) \\
\bottomrule
\end{tabular}



## Data Subset Comparisons

In [49]:
subsets = {"demo_only": demographic_cols,
           "performance_only": perf_cols,
           "activity_only": activity_cols,
           "activity_and_demo": activity_cols+demographic_cols,
           "performance_and_demo": perf_cols+demographic_cols,
           "all": list(df.columns)
          }

In [51]:
if not os.path.exists(f"../results/{dataset_name}/{experiment_name}/results_subsets.pickle"):

    results_subsets = {}
    results_subsets_feature_importances = {}

    for fold in range(folds):
        results_subsets[fold] = {}
        results_subsets_feature_importances[fold] = {}
        # Create baseline
        y_train = data_dict[f"y_train_{fold}"]
        y_val = data_dict[f"y_val_{fold}"]
        y_test = data_dict[f"y_test_{fold}"]
        y_train_val = np.concatenate([y_train,y_val])

        max_ = np.argmax(np.unique(y_train_val,return_counts=True)[1])
        y_train_val_pred_base = np.ones(y_train_val.shape[0])*max_
        y_test_pred_base = np.ones(y_test.shape[0])*max_

        results_encodings[fold]["Baseline"] = {}
        eval_res_train = get_metrics(y_train_val, y_train_val_pred_base, target=target)
        for metric in eval_res_train.keys():
            results_encodings[fold]["Baseline"][metric + " Train"] = eval_res_train[metric]
        eval_res_test = get_metrics(y_test, y_test_pred_base, target=target)
        for metric in eval_res_test.keys():
            results_encodings[fold]["Baseline"][metric + " Test"] = eval_res_test[metric]


        for subset_key in subsets:
            print(f"Preparing results for fold {fold}, subset={subset_key}")
            # Retrieve data
            z_cols = data_dict["z_cols"]

            X_train = data_dict[f"X_train_{fold}"]
            y_train = data_dict[f"y_train_{fold}"]

            X_val = data_dict[f"X_val_{fold}"]
            y_val = data_dict[f"y_val_{fold}"]

            X_test = data_dict[f"X_test_{fold}"]
            y_test = data_dict[f"y_test_{fold}"]
        
            y_train_val = np.concatenate([y_train,y_val])

            # Define data subset for LR
            z_glmm_encoded_train = data_dict[f"z_glmm_encoded_train_{fold}"] 
            z_glmm_encoded_val = data_dict[f"z_glmm_encoded_val_{fold}"] 
            z_glmm_encoded_test = data_dict[f"z_glmm_encoded_test_{fold}"] 
            X_train_lr = pd.concat([X_train,z_glmm_encoded_train],axis=1)
            X_val_lr = pd.concat([X_val,z_glmm_encoded_val],axis=1)
            X_test_lr = pd.concat([X_test,z_glmm_encoded_test],axis=1)      
            X_train_val_lr = pd.concat([X_train_lr,X_val_lr])

            # Define data subset for XGB
            z_ordinal_encoded_train = data_dict[f"z_ordinal_encoded_train_{fold}"] 
            z_ordinal_encoded_val = data_dict[f"z_ordinal_encoded_val_{fold}"] 
            z_ordinal_encoded_test = data_dict[f"z_ordinal_encoded_test_{fold}"] 
            X_train_xgb = pd.concat([X_train,z_ordinal_encoded_train],axis=1)
            X_val_xgb = pd.concat([X_val,z_ordinal_encoded_val],axis=1)
            X_test_xgb = pd.concat([X_test,z_ordinal_encoded_test],axis=1)
            X_train_val_xgb = pd.concat([X_train_xgb,X_val_xgb])


            # Define data subset for evaluation
            X_train_val_lr = X_train_val_lr[[i for i in X_train_val_lr.columns if any([j in i for j in subsets[subset_key]])]]
            X_test_lr = X_test_lr[[i for i in X_test_lr.columns if any([j in i for j in subsets[subset_key]])]]
            X_train_val_xgb = X_train_val_xgb[[i for i in X_train_val_xgb.columns if any([j in i for j in subsets[subset_key]])]]
            X_test_xgb = X_test_xgb[[i for i in X_test_xgb.columns if any([j in i for j in subsets[subset_key]])]]


            # Train base models
            res, feats = evaluate_logreg(X_train_val_lr, y_train_val, X_test_lr, y_test, target=target,tune=False, seed=RS)
            results_subsets[fold]["LR_"+subset_key] = res
            results_subsets_feature_importances[fold]["LR_"+subset_key] = feats

            res, feats = evaluate_xgb(X_train_val_xgb, y_train_val, X_test_xgb, y_test, target, tune=False, max_evals=max_evals, early_stopping_rounds=early_stopping_rounds, seed=RS)
            results_subsets[fold]["XGB_"+subset_key] = res
            results_subsets_feature_importances[fold]["XGB_"+subset_key] = feats

            # Train tuned models
            res, feats = evaluate_logreg(X_train_val_lr, y_train_val, X_test_lr, y_test, target=target, max_evals=max_evals, tune=True, seed=RS)
            results_subsets[fold]["LR_"+subset_key+"_tuned"] = res
            results_subsets_feature_importances[fold]["LR_"+subset_key+"_tuned"] = feats

            res, feats = evaluate_xgb(X_train_val_xgb, y_train_val, X_test_xgb, y_test, target, tune=True, max_evals=max_evals, early_stopping_rounds=early_stopping_rounds, seed=RS)
            results_subsets[fold]["XGB_"+subset_key+"_tuned"] = res
            results_subsets_feature_importances[fold]["XGB_"+subset_key+"_tuned"] = feats
    
    if not os.path.exists(f"../results/{dataset_name}/{experiment_name}"):
        os.makedirs(f"../results/{dataset_name}/{experiment_name}")
    with open(f"../results/{dataset_name}/{experiment_name}/results_subsets.pickle", 'wb') as handle:
        pickle.dump(results_subsets, handle, protocol=pickle.HIGHEST_PROTOCOL)
    with open(f"../results/{dataset_name}/{experiment_name}/results_subsets_feature_importances.pickle", 'wb') as handle:
        pickle.dump(results_subsets_feature_importances, handle, protocol=pickle.HIGHEST_PROTOCOL)

else:
    with open(f"../results/{dataset_name}/{experiment_name}/results_subsets.pickle", 'rb') as handle:
        results_subsets = pickle.load(handle)
    with open(f"../results/{dataset_name}/{experiment_name}/results_subsets_feature_importances.pickle", 'rb') as handle:
        results_subsets_feature_importances = pickle.load(handle)
        
        
results_subsets_df = pd.DataFrame(results_subsets[0]).transpose().sort_values("F1 Test",ascending=False).round(4)
results_subsets_df[["Accuracy Train", "F1 Train", "AUROC Train", "Accuracy Test", "F1 Test", "AUROC Test"]].style.highlight_max(color = 'lightgreen', axis = 0)

Preparing results for fold 0, subset=demo_only
SCORE: 0.5997530764282681                                                                                              
SCORE: 0.599757334248441                                                                                               
SCORE: 0.5997585094362267                                                                                              
SCORE: 0.5997575681963878                                                                                              
SCORE: 0.5998072402564697                                                                                              
100%|██████████████████████████████████████████████████| 5/5 [00:00<00:00,  9.80trial/s, best loss: 0.5997530764282681]
The best hyperparameters are :  

{'C': 0.8723696155139968}
Default performance on Test: 0.6131218793938061
SCORE: 0.6040828138937523                                                                                              
SCORE

100%|█████████████████████████████████████████████████| 5/5 [00:24<00:00,  4.81s/trial, best loss: 0.42897914146194155]
The best hyperparameters after step 3 are :  

{'learning_rate': 0.07962946763929467, 'n_estimators': 290.0, 'seed': 0, 'max_depth': 6.0, 'min_child_weight': 0.0, 'colsample_bytree': 0.8871226731983411, 'subsample': 0.881874405691565}
Test Performance after third tuning round: 0.41191625253215597
SCORE: 0.46393554032968537                                                                                             
SCORE: 0.456348230475644                                                                                               
SCORE: 0.4524504744119887                                                                                              
SCORE: 0.4654499841343897                                                                                              
SCORE: 0.4659927818832427                                                                             

KeyboardInterrupt: 

In [ ]:
results_subsets_df = pd.DataFrame(results_subsets[1]).transpose().sort_values("F1 Test",ascending=False).round(4)
results_subsets_df[["Accuracy Train", "F1 Train", "AUROC Train", "Accuracy Test", "F1 Test", "AUROC Test"]].style.highlight_max(color = 'lightgreen', axis = 0)

### Effectiveness of Parameter Tuning

In [ ]:
models = results_subsets[0].keys()
metric = "F1 Test"

subsets_folds_df = pd.DataFrame([pd.DataFrame(results_subsets[fold_num]).loc[metric,models] for fold_num in results_subsets.keys()],index=results_subsets.keys())
subsets_mean_df = subsets_folds_df.mean(axis=0)
subsets_std_df = subsets_folds_df.std(axis=0)

methods = sorted(list(subsets_mean_df.index))[1:]
not_tuned = ["Baseline"]+methods[::2]
tuned = ["Baseline"]+methods[1::2]

res_df_tune_comp_mean = pd.DataFrame([subsets_mean_df.loc[not_tuned].values,subsets_mean_df.loc[tuned]],index=["Untuned","Tuned"],columns=not_tuned).transpose()
res_df_tune_comp_std = pd.DataFrame([subsets_std_df.loc[not_tuned].values,subsets_std_df.loc[tuned]],index=["Untuned","Tuned"],columns=not_tuned).transpose()

res_df_tune_comp_mean.round(2).style.highlight_max(color = 'lightgreen', axis = 1)

In [ ]:
latex_df = res_df_tune_comp_mean.round(2).astype(str) + " (" +  res_df_tune_comp_std.round(3).astype(str) + ")"
latex_df

In [ ]:
res_df_tune_comp_diff = res_df_tune_comp_mean[["Tuned"]]-res_df_tune_comp_mean[["Untuned"]].values
res_df_tune_comp_diff.round(2)

In [ ]:
res_df_tune_comp_diff_lr = res_df_tune_comp_diff.loc[[i for i in res_df_tune_comp_diff.index if (i == "Baseline" or "LR" in i)]]
res_df_tune_comp_diff_xgb = res_df_tune_comp_diff.loc[[i for i in res_df_tune_comp_diff.index if (i == "Baseline" or "XGB" in i)]]

res_df_tune_comp_diff_lr.index = [i[3:] if i!="Baseline" else "Baseline" for i in res_df_tune_comp_diff_lr.index]
res_df_tune_comp_diff_xgb.index = [i[4:] if i!="Baseline" else "Baseline" for i in res_df_tune_comp_diff_xgb.index]
res_df_tune_comp_diff_xgb

In [ ]:
latex_df_diff = pd.concat([res_df_tune_comp_diff_lr,res_df_tune_comp_diff_xgb],axis=1)
latex_df_diff.columns = ["LR", "XGB"]
latex_df_diff

In [ ]:
print(latex_df_diff.round(2).to_latex())

### Performance Results

In [ ]:
# For LR
models = ["Baseline"]+[i for i in results_subsets[0].keys() if ("tuned" in i and "LR" in i)]
metric = "F1 Test"

#####
dataset_res_dict = {}
best_models = {}
t_test_results = {}

use_df = pd.DataFrame([pd.DataFrame(results_subsets[fold_num]).loc[metric,models] for fold_num in results_subsets.keys()],index=results_subsets.keys())

df_mean = pd.DataFrame((use_df).mean(axis=0).round(3).astype(str) + " (" + use_df.std(axis=0).round(3).astype(str) + ")").transpose()
model_dict = {i: df_mean[i].values[0] for i in df_mean.columns}

best_model = use_df.columns[use_df.mean(axis=0).argmax()]

t_test_res = np.array([stats.ttest_rel(use_df[best_model].values, use_df[model].values)[1] for model in models]).round(3)
t_test_res[np.isnan(t_test_res)] = 1.
    
res_df_lr = pd.DataFrame([model_dict])

def negative_bold(val):
    i = np.where(val.name==np.array(models))[0][0]
    return ["font-weight: bold"  if t_test_res[i]>=0.05 else "" for dataset_name in val.keys()]
    # Case without transpose:
#     return ["font-weight: bold"  if t_test_results[val.name][i]>=0.05 else "" for i in range(len(val))]

res_df_lr.style.apply(negative_bold)


In [ ]:
# For XGB
models = ["Baseline"]+[i for i in results_subsets[0].keys() if ("tuned" in i and "XGB" in i)]
metric = "F1 Test"

#####
dataset_res_dict = {}
best_models = {}
t_test_results = {}

use_df = pd.DataFrame([pd.DataFrame(results_subsets[fold_num]).loc[metric,models] for fold_num in results_subsets.keys()],index=results_subsets.keys())

df_mean = pd.DataFrame((use_df).mean(axis=0).round(3).astype(str) + " (" + use_df.std(axis=0).round(3).astype(str) + ")").transpose()
model_dict = {i: df_mean[i].values[0] for i in df_mean.columns}

best_model = use_df.columns[use_df.mean(axis=0).argmax()]

t_test_res = np.array([stats.ttest_rel(use_df[best_model].values, use_df[model].values)[1] for model in models]).round(3)
t_test_res[np.isnan(t_test_res)] = 1.
    
res_df_xgb = pd.DataFrame([model_dict])

def negative_bold(val):
    i = np.where(val.name==np.array(models))[0][0]
    return ["font-weight: bold"  if t_test_res[i]>=0.05 else "" for dataset_name in val.keys()]
    # Case without transpose:
#     return ["font-weight: bold"  if t_test_results[val.name][i]>=0.05 else "" for i in range(len(val))]

res_df_xgb.style.apply(negative_bold)


In [ ]:
res_df_lr.columns = [i[3:-6] if i != "Baseline" else "Baseline" for i in res_df_lr.columns]    
res_df_xgb.columns = [i[4:-6] if i != "Baseline" else "Baseline" for i in res_df_xgb.columns]    

latex_df_subsets = pd.concat([res_df_lr,res_df_xgb],axis=0)
latex_df_subsets.index = ["LR", "XGB"]
latex_df_subsets

In [ ]:
print(latex_df_subsets.round(2).transpose().to_latex())


### Feature Importance

In [ ]:
# top_10_importances = {}

# for model in list(results_subsets_feature_importances[fold].keys()):
#     imp_df = pd.concat([results_subsets_feature_importances[fold][model] for fold in range(folds)],axis=1)

#     if "LR" in model:
#         direction = imp_df.apply(lambda x: np.sign(x))
#         imp_df = imp_df.abs()

#     imp_df = imp_df/imp_df.sum(axis=0)

#     mean_imp_df = imp_df.mean(axis=1)
#     std_imp_df = imp_df.std(axis=1)

#     mean_imp_df = mean_imp_df.sort_values(ascending=False)
#     std_imp_df = std_imp_df.loc[mean_imp_df.index]
#     final_imps = mean_imp_df[:10]
#     final_imps["Rest"] = sum(mean_imp_df[10:])
#     top_5_importances[model] = np.array([final_imps.index.values, final_imps.values])

In [ ]:
demo_importances = {}
demo_importances_stds = {}

for model in list(results_subsets_feature_importances[fold].keys()):
    if "demo" in model or "all" in model:
        imp_df_all = pd.concat([results_subsets_feature_importances[fold][model] for fold in range(folds)],axis=1)
        
        if "LR" in model:
            direction = imp_df_all.apply(lambda x: np.sign(x))
            imp_df_all = imp_df_all.abs()
        if imp_df_all.sum().sum()!=0:
            imp_df = imp_df_all/imp_df_all.sum(axis=0)
        imp_df = imp_df.fillna(1/imp_df.shape[0])
#         imp_df = imp_df.loc[demographic_cols]

#         mean_imp_df = imp_df.mean(axis=1)
#         std_imp_df = imp_df.std(axis=1)

#         mean_imp_df = mean_imp_df.sort_values(ascending=False)
#         std_imp_df = std_imp_df.loc[mean_imp_df.index]
#         final_imps = mean_imp_df#[:10]
#         final_imps["Rest"] = sum(mean_imp_df[10:])
#         final_imps["Total"] = sum(mean_imp_df)
        demo_importances[model] = np.round(np.mean(imp_df.loc[[i for i in imp_df.index if any([j in i for j in demographic_cols])]].sum(axis=0)),2)#final_imps.values
        demo_importances_stds[model] = np.round(np.std(imp_df.loc[[i for i in imp_df.index if any([j in i for j in demographic_cols])]].sum(axis=0)),2)#final_imps.values


In [ ]:
lr_demo_imp = pd.Series({i: demo_importances[i] for i in demo_importances if "LR" in i and "tuned" in i})
xgb_demo_imp = pd.Series({i: demo_importances[i] for i in demo_importances if "XGB" in i and "tuned" in i})
lr_demo_imp.index = [i[3:-6] for i in lr_demo_imp.index]    
xgb_demo_imp.index = [i[4:-6] for i in xgb_demo_imp.index]    

lr_demo_imp_stds = pd.Series({i: demo_importances_stds[i] for i in demo_importances_stds if "LR" in i and "tuned" in i})
xgb_demo_imp_stds = pd.Series({i: demo_importances_stds[i] for i in demo_importances_stds if "XGB" in i and "tuned" in i})
lr_demo_imp_stds.index = [i[3:-6] for i in lr_demo_imp_stds.index]    
xgb_demo_imp_stds.index = [i[4:-6] for i in xgb_demo_imp_stds.index]    


latex_df_imp = pd.DataFrame([lr_demo_imp.astype(str) + " (" + lr_demo_imp_stds.astype(str) + ")",
                             xgb_demo_imp.astype(str) + " (" + xgb_demo_imp_stds.astype(str) + ")"])
latex_df_imp.index = ["LR", "XGB"]
latex_df_imp

In [ ]:
print(latex_df_subsets.to_latex())